In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
drive.mount('/gdrive')
%cd /gdrive/MyDrive/
#%ls

Mounted at /gdrive
/gdrive/MyDrive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#np.set_printoptions(formatter={'float_kind':'{:f}'.format})

#### Importar data total

In [ ]:
df = pd.read_csv("train.csv")
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


#### Primera revisión de la data completa

Valores duplicados

In [ ]:
duplicateRowsDF = df[df.duplicated()]
duplicateRowsDF

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count


Nan

In [ ]:
for i in range(df.shape[1]):
  print(df.iloc[:,[i]].isna().sum())

key    0
dtype: int64
fare_amount    0
dtype: int64
pickup_datetime    0
dtype: int64
pickup_longitude    0
dtype: int64
pickup_latitude    0
dtype: int64
dropoff_longitude    376
dtype: int64
dropoff_latitude    376
dtype: int64
passenger_count    0
dtype: int64


In [ ]:
nans_indices = df.columns[df.isna().any()].tolist()
nans = df.loc[:,nans_indices]

In [ ]:
nans

,dropoff_longitude,dropoff_latitude
0,-73.841610,40.712278
1,-73.979268,40.782004
2,-73.991242,40.750562
3,-73.991567,40.758092
4,-73.956655,40.783762
...,...,...
55423851,-73.963280,40.762555
55423852,-73.951640,40.773959
55423853,-73.960537,40.797342
55423854,-73.870605,40.773963


Valores duplicados por columna

In [ ]:
duplicateRowsDF = df[df.duplicated(['key'], keep= False)]

In [ ]:
duplicateRowsDF

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
3583097,2014-12-12 11:33:00.00000015,10.0,2014-12-12 11:33:00 UTC,-73.955212,40.773483,-73.983742,40.782178,1
55423855,2014-12-12 11:33:00.00000015,7.5,2014-12-12 11:33:00 UTC,-73.969722,40.797668,-73.970885,40.783313,1


#### Creación de una muestra al 20%

In [ ]:
dfs = df.sample(frac=0.2).sort_index()
print(dfs.shape[0])

11084771


In [ ]:
dfs.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
7,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1
16,2014-02-19 07:22:00.00000074,12.5,2014-02-19 07:22:00 UTC,-73.986430,40.760465,-73.988990,40.737075,1
21,2013-02-12 12:15:46.0000002,11.5,2013-02-12 12:15:46 UTC,-73.957954,40.779252,-73.961250,40.758787,1


In [ ]:
dfs.to_csv('trainsample.csv')

#### Importar la muestra

In [ ]:
dfs = pd.read_csv('trainsample.csv')
dfs.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
2,7,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1
3,16,2014-02-19 07:22:00.00000074,12.5,2014-02-19 07:22:00 UTC,-73.986430,40.760465,-73.988990,40.737075,1
4,21,2013-02-12 12:15:46.0000002,11.5,2013-02-12 12:15:46 UTC,-73.957954,40.779252,-73.961250,40.758787,1


In [ ]:
dfs.shape[0]

11084771

In [ ]:
dfs = dfs.drop(['Unnamed: 0'], axis=1)

#### Primera revisión de la muestra

Nan

In [ ]:
dfs.isna().sum()

key                   0
fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude    73
dropoff_latitude     73
passenger_count       0
dtype: int64

Evaluar tipos de datos por columna

In [ ]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11084771 entries, 0 to 11084770
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 676.6+ MB


Convertir `pickup_datetime` a formato datetime.

In [ ]:
dfs['pickup_datetime'] = pd.to_datetime(dfs['pickup_datetime'], format="%Y-%m-%d %H:%M:%S UTC")
dfs.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1
1,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2
2,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00,-73.951300,40.774138,-73.990095,40.751048,1
3,2014-02-19 07:22:00.00000074,12.5,2014-02-19 07:22:00,-73.986430,40.760465,-73.988990,40.737075,1
4,2013-02-12 12:15:46.0000002,11.5,2013-02-12 12:15:46,-73.957954,40.779252,-73.961250,40.758787,1


In [ ]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11084771 entries, 0 to 11084770
Data columns (total 8 columns):
 #   Column             Dtype         
---  ------             -----         
 0   key                object        
 1   fare_amount        float64       
 2   pickup_datetime    datetime64[ns]
 3   pickup_longitude   float64       
 4   pickup_latitude    float64       
 5   dropoff_longitude  float64       
 6   dropoff_latitude   float64       
 7   passenger_count    int64         
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 676.6+ MB


Preprocesamiento

In [ ]:
from sklearn import preprocessing
from sklearn import metrics
labelEncoder = preprocessing.LabelEncoder()

dfs['key']=labelEncoder.fit_transform(dfs['key'])

Buscamos datos repetidos en la data

In [ ]:
duplicateRowsDF = dfs[dfs.duplicated(keep= False)]
print(duplicateRowsDF.shape[0])

0


In [ ]:
duplicateRowsDF = dfs[dfs.duplicated(['key'], keep= False)]
print(duplicateRowsDF.shape[0])

0


Como no hay valores repetidos en la muestra, se analizará la data con el objetivo de eliminar las filas con coordenadas ubicadas en el océano.


In [ ]:
!pip install global-land-mask

     |████████████████████████████████| 1.8MB 5.1MB/s 


In [ ]:
from global_land_mask import globe as gl

In [ ]:
dfs = dfs.dropna()

In [ ]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11084698 entries, 0 to 11084770
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 761.1+ MB


In [ ]:
#dfs.drop(dfs[['pickup_latitude']].any().values<-90 or dfs[['pickup_latitude']].any().values>90)

dfs = dfs[dfs['pickup_latitude'].values > -90]
dfs = dfs[dfs['pickup_latitude'].values < 90]
dfs = dfs[dfs['dropoff_latitude'].values > -90]
dfs = dfs[dfs['dropoff_latitude'].values < 90]

dfs = dfs[dfs['pickup_longitude'].values > -180]
dfs = dfs[dfs['pickup_longitude'].values < 180]
dfs = dfs[dfs['dropoff_longitude'].values > -180]
dfs = dfs[dfs['dropoff_longitude'].values < 180]

dfs

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
2,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1
3,2014-02-19 07:22:00.00000074,12.5,2014-02-19 07:22:00 UTC,-73.986430,40.760465,-73.988990,40.737075,1
4,2013-02-12 12:15:46.0000002,11.5,2013-02-12 12:15:46 UTC,-73.957954,40.779252,-73.961250,40.758787,1
...,...,...,...,...,...,...,...,...
11084766,2009-08-27 18:39:00.00000067,11.3,2009-08-27 18:39:00 UTC,-73.980997,40.762858,-73.990503,40.736850,1
11084767,2015-05-17 00:27:40.0000005,5.0,2015-05-17 00:27:40 UTC,-73.974289,40.761883,-73.982964,40.756119,3
11084768,2009-02-03 00:28:08.0000001,10.2,2009-02-03 00:28:08 UTC,-73.999809,40.726766,-73.979344,40.749732,2
11084769,2014-09-22 09:00:00.00000019,16.5,2014-09-22 09:00:00 UTC,-73.977880,40.763550,-73.974020,40.743170,6


In [ ]:
pla = dfs[['pickup_latitude']].to_numpy()
plon = dfs[['pickup_longitude']].to_numpy()
dla = dfs[['dropoff_latitude']].to_numpy()
dlon = dfs[['dropoff_longitude']].to_numpy()

#gl.is_ocean(pla,plon)
#dfs.loc[gl.is_ocean(pla,plon), :]

dfs = dfs.drop(dfs.loc[gl.is_ocean(pla,plon)].index)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
2,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1
3,2014-02-19 07:22:00.00000074,12.5,2014-02-19 07:22:00 UTC,-73.986430,40.760465,-73.988990,40.737075,1
4,2013-02-12 12:15:46.0000002,11.5,2013-02-12 12:15:46 UTC,-73.957954,40.779252,-73.961250,40.758787,1
...,...,...,...,...,...,...,...,...
11084766,2009-08-27 18:39:00.00000067,11.3,2009-08-27 18:39:00 UTC,-73.980997,40.762858,-73.990503,40.736850,1
11084767,2015-05-17 00:27:40.0000005,5.0,2015-05-17 00:27:40 UTC,-73.974289,40.761883,-73.982964,40.756119,3
11084768,2009-02-03 00:28:08.0000001,10.2,2009-02-03 00:28:08 UTC,-73.999809,40.726766,-73.979344,40.749732,2
11084769,2014-09-22 09:00:00.00000019,16.5,2014-09-22 09:00:00 UTC,-73.977880,40.763550,-73.974020,40.743170,6


In [ ]:
# dfs[['pickup_latitude', 'pickup_longitude']] = dfs.apply(lambda x: ocean_values_to_nan(x['pickup_latitude'],x['pickup_longitude']), axis=1 )
# dfs[['dropoff_latitude', 'dropoff_longitude']] = dfs.apply(lambda x: ocean_values_to_nan(x['dropoff_latitude'],x['dropoff_longitude']) , axis=1)

In [ ]:
dfs.isna().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [ ]:
dfs.to_csv('trainsampleprep.csv')

In [ ]:
dfs.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
2,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1
3,2014-02-19 07:22:00.00000074,12.5,2014-02-19 07:22:00 UTC,-73.986430,40.760465,-73.988990,40.737075,1
4,2013-02-12 12:15:46.0000002,11.5,2013-02-12 12:15:46 UTC,-73.957954,40.779252,-73.961250,40.758787,1


Normalización de los datos para el Knn

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dfn = pd.DataFrame(scaler.fit_transform(dfs), columns = dfs.columns)
dfn.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,0.096626,0.256750,0.094779,0.486964,0.509565,0.487987,0.520137,0.004808
1,0.095404,0.254083,0.093577,0.486967,0.509570,0.487992,0.520145,0.014423
2,0.193632,0.259417,0.189024,0.486962,0.509562,0.487989,0.520142,0.004808
3,0.841936,0.255000,0.810950,0.486965,0.509566,0.487990,0.520143,0.009615
4,0.384110,0.260083,0.370704,0.486959,0.509561,0.487990,0.520142,0.004808


In [ ]:
dff = pd.DataFrame(af, columns = ['key','fare_amount','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count'])
dff.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,0.096626,0.256750,0.094779,0.486964,0.509565,0.487987,0.520137,0.004808
1,0.095404,0.254083,0.093577,0.486967,0.509570,0.487992,0.520145,0.014423
2,0.193632,0.259417,0.189024,0.486962,0.509562,0.487989,0.520142,0.004808
3,0.841936,0.255000,0.810950,0.486965,0.509566,0.487990,0.520143,0.009615
4,0.384110,0.260083,0.370704,0.486959,0.509561,0.487990,0.520142,0.004808


Verificar valores Nan

In [ ]:
dff.isna().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64